In [1]:
import PI

meta_params = {
    'num_of_train_dataset': 1000,
    'num_of_test_dataset': 1000,
    'is_flatten': False
}

PI = PI.PIInterface(meta_params)

import torch
import torch.nn as nn
import torch.optim as optim
from MNIST_models import *

model = load_model('store/MNIST_CNN.pt')
PI.set_model(model)
print('train acc:', PI.eval_model('train'))
print('test acc:', PI.eval_model('test'))

1000 1000
(1000, 1, 28, 28)


/Users/kuotzuyang/opt/anaconda3/lib/python3.7/site-packages/torch/serialization.py:593: SourceChangeWarning: source code of class 'MNIST_models.CNN' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/Users/kuotzuyang/opt/anaconda3/lib/python3.7/site-packages/torch/serialization.py:593: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/Users/kuotzuyang/opt/anaconda3/lib/python3.7/site-packages/torch/serialization.py:593: SourceChangeWarning: source code of class 'torch.nn.modules.linear.Linear' has changed. you can retrieve the original source code by accessi

train acc: 0.991
test acc: 0.974


In [2]:
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

from scipy.ndimage.filters import gaussian_filter1d
from matplotlib import pyplot as plt
%matplotlib inline

prefixs = ['store_zero/', 'store_one/', 'store_two/', 'store_three/', 'store_four/', 
           'store_five/', 'store_six/', 'store_seven/', 'store_eight/', 'store_nine/']      

guards = []
for idx, prefix in enumerate(prefixs):
    # LOAD TRAINED SUB-GUARD
    guard_model = torch.load('store_subs/'+prefix+'guard.pt')
    guards.append(guard_model)
        
for idx, prefix in enumerate(prefixs):
    print(prefix)
    
    # LOAD DATASET 
    adv_types = ['None', 'FGSM', 'JSMA', 'CWL2', 'LINFPGD', 'LINFBI', 'ENL1', 'ST']
    set_of_train_dataset, set_of_test_dataset = [], []
    for adv_type in adv_types:
        # extract from the store file 
        if adv_type == 'None': fn_name = 'store_subs/'+prefix+'normal.txt'
        else: fn_name = 'store_subs/'+prefix+adv_type+'.txt'
        fp = open(fn_name, 'rb')
        set_of_signatures = pickle.load(fp)

        # separate and store in for later training and evaluation 
        if adv_type == 'None': split_percentage = 0.8
        else: split_percentage = 0.8
        split_line = int(len(set_of_signatures)*split_percentage)
        train_set_of_signatures, test_set_of_signatures = set_of_signatures[:split_line], set_of_signatures[split_line:]
        set_of_train_dataset.append(train_set_of_signatures)
        set_of_test_dataset.append(test_set_of_signatures)
        fp.close()
        print(adv_type, len(set_of_signatures), len(train_set_of_signatures), len(test_set_of_signatures))
    print()


store_zero/
None 98 78 20
FGSM 75 60 15
JSMA 93 74 19
CWL2 87 69 18
LINFPGD 92 73 19
LINFBI 98 78 20
ENL1 98 78 20
ST 98 78 20

store_one/
None 102 81 21
FGSM 94 75 19
JSMA 102 81 21
CWL2 98 78 20
LINFPGD 100 80 20
LINFBI 100 80 20
ENL1 102 81 21
ST 93 74 19

store_two/
None 105 84 21
FGSM 80 64 16
JSMA 105 84 21
CWL2 74 59 15
LINFPGD 100 80 20
LINFBI 105 84 21
ENL1 105 84 21
ST 105 84 21

store_three/
None 103 82 21
FGSM 68 54 14
JSMA 103 82 21
CWL2 78 62 16
LINFPGD 102 81 21
LINFBI 80 64 16
ENL1 103 82 21
ST 103 82 21

store_four/
None 96 76 20
FGSM 87 69 18
JSMA 94 75 19
CWL2 94 75 19
LINFPGD 96 76 20
LINFBI 96 76 20
ENL1 96 76 20
ST 96 76 20

store_five/
None 90 72 18
FGSM 84 67 17
JSMA 90 72 18
CWL2 83 66 17
LINFPGD 90 72 18
LINFBI 90 72 18
ENL1 90 72 18
ST 90 72 18

store_six/
None 107 85 22
FGSM 101 80 21
JSMA 106 84 22
CWL2 80 64 16
LINFPGD 107 85 22
LINFBI 104 83 21
ENL1 107 85 22
ST 107 85 22

store_seven/
None 107 85 22
FGSM 91 72 19
JSMA 107 85 22
CWL2 100 80 20
LINFPGD 105

In [3]:
adv_types = ['None', 'FGSM', 'JSMA', 'CWL2', 'LINFPGD', 'LINFBI', 'ENL1', 'ST']
# adv_types = ['None', 'FGSM']
for adv_type in adv_types:
    print(adv_type)
    if adv_type == 'None': 
        PI.eval_sub_guards(guards)
    else:
        PI.eval_sub_guards(guards, adv_type)
    print()

None
correct_count: 861 total_count: 974
acc: 0.8839835728952772

FGSM
correct_count: 723 total_count: 816
acc: 0.8860294117647058

JSMA
correct_count: 924 total_count: 973
acc: 0.9496402877697842

CWL2
correct_count: 766 total_count: 860
acc: 0.8906976744186047

LINFPGD
correct_count: 560 total_count: 973
acc: 0.5755395683453237

LINFBI
correct_count: 423 total_count: 957
acc: 0.44200626959247646

ENL1
correct_count: 894 total_count: 1000
acc: 0.894

ST
correct_count: 935 total_count: 988
acc: 0.9463562753036437

